# Filter bam files on quality 20

In [ ]:
q20=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20
bams=$(ls /workspace/hramzr/2_Phd_PROJECT/WGS/IR/*.bam)
module load samtools/1.7
for bam in $bams
do
prefix=$(basename $bam | awk -F "_" '{print $1}')
echo "samtools view -hb -q 20 $bam -o ${q20}/${prefix}_q20.bam"
done | asub -j qfilter

# Index q20 bamfiles

In [ ]:
module load samtools/1.7
for bam in $(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*.bam)
do
    echo "samtools index $bam"
done | asub -j index

# Get Parliament2 ensemble caller Docker

In [ ]:
# download docker for singularity
module load singularity/2.6.1
# singularity pull docker://sameerdcosta/parliament2
bsub \
-J "pull singularity docker" \
"singularity pull --name parliament2.simg docker://dnanexus/parliament2:latest"

# Make separate directionaries for parliament

In [ ]:
bams=$(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*.bam)
calldir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling
for bam in $bams
do
prefix=$(basename $bam | awk -F "_" '{print $1}')
mkdir ${calldir}/$prefix
mkdir ${calldir}/${prefix}/outputs
done

# Load bam, index and reference files into parliament directionaries

In [ ]:
PATH=$(getconf PATH)
echo $PATH
python -v

In [ ]:
bams=$(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*967038331*.bam)
calldir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/
ref=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta 
fai=${ref}.fai
log=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/logs
for bam in $bams
do
prefix=$(basename $bam | awk -F "_" '{print $1}')
bai=$(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/${prefix}*.bai)
# echo $bai
# echo $bam
# echo $prefix
# echo $fai
# echo $ref
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp $bam ${calldir}${prefix}/"
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp $bai ${calldir}${prefix}/"
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp $ref ${calldir}${prefix}/"
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp $fai ${calldir}${prefix}/"
bsub -o ${log}/cp.o -e ${log}/cp.e \
"cp /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2.simg ${calldir}${prefix}/"
done

# Run parliament2

In [ ]:
python --version

In [ ]:
module load singularity/2.6.1
bams=$(ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*967038331*.bam)
wdir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/
out=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_out
indir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/
ref=Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta
fai=Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta.fai
log=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/logs/
for bam in $bams
do
prefix=$(basename $bam | awk -F "_" '{print $1}')
cd ${wdir}${prefix}
bsub \
-o ${log}sing.out -e ${log}sing.err -J "sing" \
-R "rusage[mem=64000] span[hosts=1]" \
"singularity run -B ${wdir}${prefix}:/home/dnanexus/in,${wdir}${prefix}/outputs:/home/dnanexus/out parliament2.simg \
--bam ${prefix}_q20.bam --bai ${prefix}_q20.bam.bai --fai $fai -r $ref \
    --filter_short_contigs \
    --breakdancer \
    --breakseq \
    --manta \
    --cnvnator \
    --lumpy \
    --delly_deletion \
    --delly_insertion \
    --delly_inversion \
    --delly_duplication \
    --genotype"
done 

# Run FreeBayes

In [6]:
indvs=$(cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs)
for value in $indvs
do
mkdir /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/indv_dirs/${value}
done

/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/000475368_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008801547_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008808514_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008813278_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008819521_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008820870_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008821357_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008825059_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008825377_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008828555_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008828871_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008831557_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008831607_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/008833125_q20.bam
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q2

In [ ]:
module load freebayes/1.1.0
module load tabix/0.2.6
module load samtools/1.7
export TMPDIR=/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper/tmp

OUT="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes"
LOG="${OUT}/logs"
SAMPLES="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/male_female_bams.lst"
PAR="/software/bioinformatics/freebayes-1.1.0/scripts"
ref=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta 
fai=${ref}.fai
TEMP=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/tmp

bsub << EOF
#!/bin/bash
#BSUB -J "Free"
#BSUB -o ${LOG}/freeb.out 
#BSUB -e ${LOG}/freeb.err
#BSUB -n 80
#BSUB -R "span[hosts=1]"

freebayes-parallel <($PAR/fasta_generate_regions.py $fai 1000000) 80 \
             -f ${ref} \
             -L ${SAMPLES} \
             --ploidy 2 \
             --report-genotype-likelihood-max \
             --min-base-quality 10 \
             --min-mapping-quality 20 \
             --genotype-qualities \
             --use-mapping-quality \
             --no-mnps \
             --no-complex \
             --max-complex-gap 50 \
             --min-alternate-fraction 0.1 \
             --min-repeat-entropy 1 \
             --no-partial-observations \
             --min-coverage 10 \
             --max-coverage 500 \
             --pooled-continuous>${OUT}/male_female_Samples_FB.vcf
EOF

In [20]:
module load freebayes/1.1.0
module load tabix/0.2.6
module load samtools/1.7
export TMPDIR=/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper/tmp
OUT="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/indv_dirs/"
LOG="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/indv_dirs/logs"
SAMPLES="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/male_female_bams.lst"
PAR="/software/bioinformatics/freebayes-1.1.0/scripts"
ref=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fasta 
fai=${ref}.fai
TEMP=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/tmp
bamlst=$(find /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*.bam)
for value in $bamlst
do
prefix=$(echo $value | awk -F '/' '{print $NF}' | awk -F "_" '{print $1}')
bsub << EOF
#!/bin/bash
#BSUB -J "Free"
#BSUB -o ${LOG}/freeb.out 
#BSUB -e ${LOG}/freeb.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

freebayes-parallel <($PAR/fasta_generate_regions.py $fai 1000000) 80 \
             -f ${ref} ${value} \
             --ploidy 2 \
             --report-genotype-likelihood-max \
             --min-base-quality 10 \
             --min-mapping-quality 20 \
             --genotype-qualities \
             --use-mapping-quality \
             --no-mnps \
             --no-complex \
             --max-complex-gap 50 \
             --min-alternate-fraction 0.1 \
             --min-repeat-entropy 1 \
             --no-partial-observations \
             --min-coverage 10 \
             --max-coverage 500 \
             --pooled-continuous>${OUT}${prefix}/${prefix}_sample_FB.vcf
EOF
done

Job <434345> is submitted to default queue <lowpriority>.
Job <434346> is submitted to default queue <lowpriority>.
Job <434347> is submitted to default queue <lowpriority>.
Job <434348> is submitted to default queue <lowpriority>.
Job <434349> is submitted to default queue <lowpriority>.
Job <434350> is submitted to default queue <lowpriority>.
Job <434351> is submitted to default queue <lowpriority>.
Job <434352> is submitted to default queue <lowpriority>.
Job <434353> is submitted to default queue <lowpriority>.
Job <434354> is submitted to default queue <lowpriority>.
Job <434355> is submitted to default queue <lowpriority>.
Job <434356> is submitted to default queue <lowpriority>.
Job <434357> is submitted to default queue <lowpriority>.
Job <434358> is submitted to default queue <lowpriority>.
Job <434359> is submitted to default queue <lowpriority>.
Job <434360> is submitted to default queue <lowpriority>.
Job <434361> is submitted to default queue <lowpriority>.
Job <434362> i

# Sex trait search
### First we need to separate female from male samples and make a selection of an equal number of samples
#### Here that is 9 males and 9 females

In [ ]:
sex_list=/powerplant/workspace/hramzr/2_Phd_PROJECT/Genomics/Fish/Resequencing/Chrysophrys_auratus/10565/metadata/additional_metadata/2017_03_06_mb_sex_determination.txt
cat $sex_list | tail -n+2 | awk '{print $4, $5}' | egrep "female">/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/females_sexfile2017.lst
cat $sex_list | tail -n+2 | awk '{print $4, $5}' | egrep -w "male">/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/males_sexfile2017.lst
ls /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/*.bam | awk -F "/" '{print $NF}' | awk -F "_" '{print $1}'>/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs

cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/females_sexfile2017.lst | tr -d [a-zA-Z] | tr -d " " | sort | uniq -c | sort -rk1 | egrep -w "2" | \
awk '{print $2}'>/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/females.lst

cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/males_sexfile2017.lst | tr -d [a-zA-Z] | tr -d " " | sort | uniq -c | sort -rk1 | egrep -w "2" | \
awk '{print $2}'>/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/males.lst

cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/males.lst | wc -l
cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/females.lst | wc -l

In [ ]:
vcf_dir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/
female_ids=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/females.lst 
male_ids=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/males.lst

# Make number to size plots for inversions/duplications/deletions
### Start with making a grouped cat variable to get sizes and N for males and females

In [ ]:
female_ids=$(cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/females.lst) 
male_ids=$(cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/males.lst) 
vcf_dir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/

female_vcfs=""
for idf in $female_ids
do
female_vcfs+="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/${idf}/outputs/${idf}_q20.survivor_sorted.vcf "
done

male_vcfs=""
for idm in $male_ids
do
male_vcfs+="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/${idm}/outputs/${idm}_q20.survivor_sorted.vcf "
done

### Then call the amount of DEL DUP INS INV TRA for all of the female samples and male samples
#### Output is file with N and Size

In [ ]:
out=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/
#GET DELS
cat $female_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="DEL"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}female_dels.file

cat $male_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="DEL"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}male_dels.file
#GET DUPS
cat $female_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="DUP"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}female_dups.file

cat $male_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="DUP"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}male_dups.file
#GET INS
cat $female_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="INS"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}female_ins.file

cat $male_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="INS"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}male_ins.file

#GET INVS
cat $female_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="INV"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}female_invs.file

cat $male_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="INV"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}male_invs.file

#GET TRA's
cat $female_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="TRA"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}female_tras.file

cat $male_vcfs | egrep -v "#" | \
tr ";" " " | awk '{if (substr($3,1,3)=="TRA"){print substr($10,8)}}' | sort | uniq -c | sed -e 's/^[[:space:]]*//'>${out}male_tras.file


## Plot with pandas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import matplotlib.patches as mpatches

basedir='/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/'
df2=pd.read_csv(basedir+"male_dels.file", header=None, delimiter=" ")  
df=pd.read_csv(basedir+"female_dels.file", header=None, delimiter=" ")  
ax1 = df.plot(kind='hist',x=0,y=1,figsize=(20,5), bins=500) # scatter plot
ax2 = df2.plot(kind='hist',x=0,y=1, figsize=(20,5), ax=ax1, color='r', bins=500, alpha=0.5)
tt = mpatches.Patch(color='red', label='females')
ch = mpatches.Patch(color='blue', label='males')
plt.legend(handles=[tt,ch], loc=0)
ax1.set_ylabel("Size")
ax1.set_xlabel("N")
plt.yscale('log')
ax1.set_title("Histogram deletions male vs female with bin=500")
plt.show()
print (ax1==ax2)



basedir='/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/'
df2=pd.read_csv(basedir+"male_dups.file", header=None, delimiter=" ")  
df=pd.read_csv(basedir+"female_dups.file", header=None, delimiter=" ")  
ax1 = df.plot(kind='hist',x=0,y=1,figsize=(20,5), bins=500) # scatter plot
ax2 = df2.plot(kind='hist',x=0,y=1, figsize=(20,5), ax=ax1, color='r', bins=500, alpha=0.5)
tt = mpatches.Patch(color='red', label='females')
ch = mpatches.Patch(color='blue', label='males')
plt.legend(handles=[tt,ch], loc=0)
ax1.set_ylabel("Size")
ax1.set_xlabel("N")
plt.yscale('log')
ax1.set_title("Histogram dups male vs female with bin=500")
plt.show()
print (ax1==ax2)


basedir='/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/'
df2=pd.read_csv(basedir+"male_ins.file", header=None, delimiter=" ")  
df=pd.read_csv(basedir+"female_ins.file", header=None, delimiter=" ")  
ax1 = df.plot(kind='hist',x=0,y=1,figsize=(20,5), bins=500) # scatter plot
ax2 = df2.plot(kind='hist',x=0,y=1, figsize=(20,5), ax=ax1, color='r', bins=500, alpha=0.5)
tt = mpatches.Patch(color='red', label='females')
ch = mpatches.Patch(color='blue', label='males')
plt.legend(handles=[tt,ch], loc=0)
ax1.set_ylabel("Size")
ax1.set_xlabel("N")
plt.yscale('log')
ax1.set_title("Histogram ins male vs female with bin=500")
plt.show()
print (ax1==ax2)



basedir='/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/'
df2=pd.read_csv(basedir+"male_invs.file", header=None, delimiter=" ")  
df2
df=pd.read_csv(basedir+"female_invs.file", header=None, delimiter=" ")  
ax1 = df.plot(kind='hist',x=0,y=1,figsize=(20,5), bins=500) # scatter plot
ax2 = df2.plot(kind='hist',x=0,y=1, figsize=(20,5), ax=ax1, color='r', bins=500, alpha=0.5)
tt = mpatches.Patch(color='red', label='females')
ch = mpatches.Patch(color='blue', label='males')
plt.legend(handles=[tt,ch], loc=0)
ax1.set_ylabel("Size")
ax1.set_xlabel("N")
plt.yscale('log')
ax1.set_title("Histogram invs male vs female with bin=500")
plt.show()
print (ax1==ax2)


basedir='/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/'
df2=pd.read_csv(basedir+"male_tras.file", header=None, delimiter=" ")  
df2
# df2
df=pd.read_csv(basedir+"female_tras.file", header=None, delimiter=" ")  
# df
ax1 = df.plot(kind='hist',x=0,y=1,figsize=(20,5), bins=10) # scatter plot
ax2 = df2.plot(kind='hist',x=0,y=1, figsize=(20,5), ax=ax1, color='r', bins=10, alpha=0.5)
tt = mpatches.Patch(color='red', label='females')
ch = mpatches.Patch(color='blue', label='males')
plt.legend(handles=[tt,ch], loc=0)
ax1.set_ylabel("Size")
ax1.set_xlabel("N")
# plt.yscale('log')
ax1.set_title("Histogram tras male vs female with bin=500")
plt.show()
print (ax1==ax2)

## Analysis:
#### There are no distinct TRAS.
#### There are some small distinct male and female numbers and sizes in invs, dups and dels. 
#### There are clear distinct male and female numbers and sizes in ins. Worth to look more into.
## Next up: merge the variants in a unified VCF 
#### Only tool that can do this isnstead of vcf-merge is SURVIVOR https://github.com/fritzsedlazeck/SURVIVOR
#### This is due to the fact that breakpoints are estimates and thus are slightly different between samples even if they are the same variant. Merge-vcf does not account for this due to it being geared toward SNP calling.
#### We deal with this by a distance setting specifying the maximal difference between pairs of breakpoints (start1 vs start2 and end1 vs end2)
## Start: download the tool from GIT into VCFdir

In [ ]:
cd /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge
git clone https://github.com/fritzsedlazeck/SURVIVOR.git
cd /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/SURVIVOR/Debug
make

# Load and look at options

In [ ]:
/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/SURVIVOR/Debug/SURVIVOR

# Create list to merge == the 9 males and 9 females

In [ ]:
out=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/
rm ${out}mergelist
echo $male_vcfs | tr " " "\n">${out}mergelist
echo $female_vcfs | tr " " "\n">>${out}mergelist
cat ${out}mergelist



In [ ]:
female_ids=$(cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/females.lst) 
male_ids=$(cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/males.lst) 
vcf_dir=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/
female_vcfs=""
for idf in $female_ids
do
female_vcfs+="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/${idf}/outputs/${idf}_q20.combined.genotyped.vcf "
done

male_vcfs=""
for idm in $male_ids
do
male_vcfs+="/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/${idm}/outputs/${idm}_q20.combined.genotyped.vcf "
done

# Merge the files
## params = 2 callers, strand, type and 30 bp minimum size with distance breakpointpair being 500 max

In [ ]:
out=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/

outfile=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/9_male_female_merged_samples.vcf
log=//workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/logs/
bsub \
-o ${log}survivor_merge.o -e ${log}survivor_merge.e  -J "survivor merge" \
"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/SURVIVOR/Debug/SURVIVOR merge ${out}mergelist \
5000 1 1 1 0 5 \
${outfile}"

# Create genotype input for statistical parsing
## Done with VCFR

In [ ]:
#Get genotype from VCF per sample and scaffold pos
library(vcfR)
vcf <- read.vcfR("/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/id_adjusted.vcf", checkFile=F)
gt <- extract.gt(vcf, element = c('GT'))

#Extract the genotype and transform it into a table, then write out the results to a csv.
?extract.gt
library(dplyr)
library(data.table)
write.csv(gt, "/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/9_male_female_merged_genotypes.csv")

# Open with pandas

In [ ]:
import pandas as pd
df=pd.read_csv("/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/80_samples.gt.GT.FORMAT", delimiter="\t")  
df

In [ ]:
import pandas as pd

with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/9fm_samples.gt.GT.FORMAT") as f:
    next(f)
    for line in f:
        males=line.split()[2:11]
        females=line.split()[11:]
        good="yes"
        for value in males:
            if value in females:
                good="no"
        if good == "yes":
            print (line)
        


In [ ]:
indvs = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/malesnfemales", "r")
indvlist=[]
for indv in indvs:
    indvlist.append(indv.strip())
males=indvlist[0:9]
females=indvlist[9:]
indexes_m=[]
indexes_f=[]
with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/male_female_Samples_FB.vcf") as f:
    for line in f:
        if "SID" in line:
            for i in range (18):
                if  line.split()[i+9].replace("SID_", "") in males:
                    indexes_m.append(i+9)
                else:
                    indexes_f.append(i+9)

In [ ]:
print(indexes_f)
print(indexes_m)

In [ ]:
import pandas as pd
import collections
import numpy as np
import scipy.stats as stats
import collections
# chidata = open("/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/chidata", "w")    
# with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/9_male_female_merged_samples.vcf") as f:
with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/male_female_Samples_FB.vcf") as f:
    for line in f:
        if "#" not in line:
            end=line.split()[7].split(";")[6][4:]
            chrom=line.split()[0]
            start=line.split()[1]
    #         svlen=line.split(";")[2][6:]
    #         svtype=line.split()[4][1:4]
            males=line.split()[indexes_m[0]][0:3], line.split()[indexes_m[1]][0:3],line.split()[indexes_m[2]][0:3],line.split()[indexes_m[3]][0:3],line.split()[indexes_m[4]][0:3], \
            line.split()[indexes_m[5]][0:3], line.split()[indexes_m[6]][0:3], line.split()[indexes_m[7]][0:3], line.split()[indexes_m[8]][0:3]
            females=line.split()[indexes_f[0]][0:3], line.split()[indexes_f[1]][0:3], line.split()[indexes_f[2]][0:3], line.split()[indexes_f[3]][0:3], line.split()[indexes_f[4]][0:3], line.split()[indexes_f[5]][0:3], line.split()[indexes_f[6]][0:3], line.split()[indexes_f[7]][0:3], line.split()[indexes_f[8]][0:3]
            good="yes"
            for value in males:
                if value in females:
                    good="no"
            if good == "yes":
                if "0" not in males or females:
                    print (chrom, start, end)
                    print(males, females)


#             line=list(males)+list(females)
#             list_of_unique_indexes=[]
#             s = pd.Series(line)
#             s = pd.factorize(s)[0] + 1
#             final_line = s.tolist()
#             list_of_unique_indexes.append(final_line)        
#             for line in list_of_unique_indexes:    
#                 male = line[0:9]
#                 female = line[9:]
#                 varlist= []
#                 counterM=collections.Counter(male)
#                 counterF=collections.Counter(female)
#                 for i in range(max(line)):
#                     varlist.append([counterM[i+1], counterF[i+1]])
#                 if len(counterM)== 1 and len(counterF) == 1:
#                     if counterM.keys() != counterF.keys():
#                         print (chrom, start)
#                         print(males, females)
                
#                 table = np.array(varlist)

#                 chi2_stat, p_val, dof, ex = stats.chi2_contingency(table)   
#                 if p_val <0.003:
#                     print(males, females)
#                     print(str(chrom)+" "+str(svtype)+" "+str(start)+" "+str(end)+" "+str(p_val))

#             #p_value 1.0 means no difference between samples lower than 1.0 means a difference the lower the more difference

#             #write scaffold name, pos and p_value to chi_data out 
#                 chidata.write(str(chrom)+" "+str(svtype)+" "+str(start)+" "+str(end)+" "+str(p_val)+"\n")
# chidata.close()   


In [ ]:
var=$(cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/mergelist | awk -F "/" '{print $7}')
out=/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/male_female_bams.lst
rm $out
for id in $var
do
find /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/q20/ | egrep $id | egrep "*.bam$">>$out
done


In [ ]:
module load samtools/1.9
bsub \
-e /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/depth.e -o /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/depth.o -J "depth" \
"samtools depth -a -f /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/male_female_bams.lst \
 -r LG13:17985500-17987800>/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/009022893_depth_dels"

In [99]:
import pandas as pd
import collections
import re





indvs = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/malesnfemales", "r")
indvlist=[]
for indv in indvs:
    indvlist.append(indv.strip())
males=indvlist[0:9]
females=indvlist[9:]
females.append("000475368")



indvs = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs", "r")
body = open("/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/body", "w")

sex=""
for indv in indvs:

    if str(indv).strip() in males:
        sex="male"
    elif str(indv).strip() in females:
        sex="female"
    else:
        sex="unknown"        
    body.write(indv.strip()+" "+sex+" ")
    bin_size=1
    chrom="LG1"
    with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/"+ \
               str(indv.split()[0])+"/outputs/"+str(indv.split()[0])+"_q20.combined.genotyped.vcf", "r") as f:
        chrom_dict={}
        header_list=[]
        for i in range(25):
            chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0, 'TRA':0, 'BND':0}
        for line in f:
            if "#" not in line:
#                 if int(start) and int(end
                start=line.split()[1]
                if chrom != line.split()[0][3:]:
                    bin_size=1
                if ((int(start)/1000000)>bin_size):
#                     print(start)                    
                    row=str(chrom_dict[chrom])
                    row=re.sub("[^0-9]", " ", row)
                    row=' '.join(row.split())
#                     print(chrom)

                    body.write(row+" ")
                   
                    for i in range(25):
                        chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0, 'TRA':0, 'BND':0}
                    bin_size+=1
                end=line.split()[7].split(";")[6][4:]
                chrom=line.split()[0][3:]
                start=line.split()[1]
                svlen=line.split(";")[2][7:]
                svtype=line.split(";")[3][7:]
                chrom_dict[chrom][svtype]+=1
    
    body.write("\n")
body.close()

In [100]:
import pandas as pd
import collections
import re

# indvs = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs", "r")


# for indv in indvs:
#     bin_size=1
#     chrom="LG1"
#     with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/"+ \
#                str(indv.split()[0])+"/outputs/"+str(indv.split()[0])+"_q20.combined.genotyped.vcf", "r") as f:
#         chrom_dict={}
#         header_list=[]
#         for i in range(25):
#             chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0, 'TRA':0, 'BND':0}
#             header_list.append("LG"+str(i+1)+" DEL", "LG"+str(i+1)+" DUP")
#         for line in f:
#             if "#" not in line:
# #                 if int(start) and int(end
#                 start=line.split()[1]
#                 if chrom != line.split()[0][3:]:
#                     bin_size=1
#                 if ((int(start)/1000000)>bin_size):
#                     print(start)                    
#                     row=str(chrom_dict[chrom])
#                     row=re.sub("[^0-9]", " ", row)
#                     row=' '.join(row.split())
#                     print(chrom)

#                     print(row+"\n")
                   
#                     for i in range(25):
#                         chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0, 'TRA':0, 'BND':0}
#                     bin_size+=1
#                 end=line.split()[7].split(";")[6][4:]
#                 chrom=line.split()[0][3:]
#                 start=line.split()[1]
#                 svlen=line.split(";")[2][7:]
#                 svtype=line.split(";")[3][7:]
#                 chrom_dict[chrom][svtype]+=1
#     break

                    

        
        
        
        
        
indvs = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs", "r")
headers = open("/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/headers", "w")

for indv in indvs:
    bin_size=1
    chrom="LG1"
    headers.write("indv sex ")
    with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/"+ \
               str(indv.split()[0])+"/outputs/"+str(indv.split()[0])+"_q20.combined.genotyped.vcf", "r") as f:
        chrom_dict={}
        header_list=[]
        for i in range(25):
            chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0, 'TRA':0, 'BND':0}
        for line in f:
            if "#" not in line:
#                 if int(start) and int(end
                start=line.split()[1]
                if chrom != line.split()[0][3:]:
                    bin_size=1
                if ((int(start)/1000000)>bin_size):                    
                    row=str(chrom_dict[chrom])
                    row=re.sub("[^0-9]", " ", row)
                    row=' '.join(row.split())
                    headers.write(chrom+"_dels_"+str(bin_size*1000000)+" "+chrom+"_dups_"+str(bin_size*1000000)+" "+ \
                          chrom+"_ins_"+str(bin_size*1000000)+" "+ \
                          chrom+"_invs_"+str(bin_size*1000000)+" "+ \
                          chrom+"_tras_"+str(bin_size*1000000)+" " +chrom+"_bnds_"+str(bin_size*1000000)+" ")
                    for i in range(25):
                        chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0, 'TRA':0, 'BND':0}
                    bin_size+=1
                chrom=line.split()[0][3:]
                start=line.split()[1]
    break
headers.write("\n")
headers.close()
                
        

In [ ]:
import pandas as pd
import collections
import re
z = open("/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/svs_per_chrom", "w")
indvs = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs", "r")

for i in range(25):
    if i == 0:
        head=str("indv "+"sex "+"LG"+str(i+1)+"_"+"deletions"+" LG"+str(i+1)+"_"+"duplications"+ \
         " LG"+str(i+1)+"_"+"insertions"+" LG"+str(i+1)+"_"+"inversions"+ \
         " LG"+str(i+1)+"_"+"Translocations"+" LG"+str(i+1)+"_"+"BNDs"+" LG"+str(i+1)+"_"+"sv_length_avg ")
    else:
        head=str("LG"+str(i+1)+"_"+"deletions"+" LG"+str(i+1)+"_"+"duplications"+ \
         " LG"+str(i+1)+"_"+"insertions"+" LG"+str(i+1)+"_"+"inversions"+ \
         " LG"+str(i+1)+"_"+"Translocations"+" LG"+str(i+1)+"_"+"BNDs"+" LG"+str(i+1)+"_"+"sv_length_avg ")

    z.write(head)
sex=0
for indv in indvs:
    
    with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/"+ \
               str(indv.split()[0])+"/outputs/"+str(indv.split()[0])+"_q20.survivor_sorted.vcf", "r") as f:
        next(f)
        chrom_dict={}
        sv_count={}
        for i in range(25):
            chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0, 'TRA':0, 'BND':0, 'sv_length_avg':0}
            sv_count["LG"+str(i+1)]=0
        for line in f:
            if "#" not in line:
                end=line.split()[7].split(";")[6][4:]
                chrom=line.split()[0]
                start=line.split()[1]
                svlen=line.split(";")[2][7:]
                svtype=line.split(";")[3][7:]
                chrom_dict[chrom][svtype]+=1
                chrom_dict[chrom]['sv_length_avg']+=round(float(svlen))
                sv_count[chrom]+=1
        z.write("\n")
        if sex <9:
            assigned_sex="1"
        elif sex <18:
            assigned_sex="0"
        else:
            assigned_sex="2"
        for i in range(25):
#             print(int(chrom_dict["LG"+str(i+1)]['sv_length_avg']/sv_count["LG"+str(i+1)]))
            chrom_dict["LG"+str(i+1)]['sv_length_avg']=int(chrom_dict["LG"+str(i+1)]['sv_length_avg']/sv_count["LG"+str(i+1)])
            if i ==0:
                line=str(indv)+" "+assigned_sex+" "+str(chrom_dict["LG"+str(i+1)]).replace("{", "").replace("}", "").replace(":", "")
            else:
                line=str(chrom_dict["LG"+str(i+1)]).replace("{", "").replace("}", "").replace(":", "")
            row=re.sub("[^0-9]", " ", line)
            row=' '.join(row.split())
            print(row)
            z.write(row+" ")  
    sex +=1
z.close()




#             break
#             males=line.split()[9][0:3], line.split()[10][0:3],line.split()[11][0:3],line.split()[12][0:3],line.split()[13][0:3], \
#             line.split()[14][0:3], line.split()[15][0:3], line.split()[16][0:3], line.split()[17][0:3]
#             females=line.split()[18][0:3], line.split()[19][0:3], line.split()[20][0:3], line.split()[21][0:3], line.split()[22][0:3], \
#             line.split()[23][0:3], line.split()[24][0:3], line.split()[25][0:3], line.split()[26][0:3]
#             if counterM["./."]>0 and counterF["./."]<0 or counterM["./."]<0 and counterF["./."]>0:
#                 print(chrom, start, end, svtype, abs(int(svlen)))
#                 print(males, females)
                

In [ ]:
cat svs_per_chrom | tr "\n" "-" | tr -s [[:space:]] "," | tr "-" "\n"

In [ ]:
import pandas as pd
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import svm
import warnings
df=pd.read_csv("/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/svs_per_chrom", delimiter=" ")  
df=df.loc[:, df.columns != 'Unnamed: 177']
df
# y = df.sex
# x = df.drop(columns=["sex"])
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

# x
# ranfor = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 0, max_features = 'auto', max_depth = 1000)
# ranfor.fit(x_train, y_train)
# pred_ranfor = ranfor.predict(x_test)
# print(accuracy_score(y_test, pred_ranfor))
# pred_ranfor

In [ ]:
module avail

In [ ]:
module load vcftools/0.1.14

# rm id_adjusted.vcf
# cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/9_male_female_merged_samples.vcf \
# | egrep "#">id_adjusted.vcf
# cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/9_male_female_merged_samples.vcf \
# | egrep -v "#" | awk -F ";" '{print substr($7,5)"_"$0}' | tr " " "\t">>id_adjusted.vcf


vcftools --vcf /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/9_male_female_merged_samples.vcf --extract-FORMAT-info GT --out /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/9fm_samples.gt
# vcftools --vcf /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/freebayes/80_Samples_FB.vcf --extract-FORMAT-info GT --out /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/vcfmerge/80_samples.gt


In [ ]:
cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/275378212/outputs/275378212_q20.survivor_sorted.vcf | egrep -v "#" | head

## -
## -
## -
## -
## -
## -
## -

# Population metrics 9 male 9 female pool of samples

In [ ]:
mlist=$(cat ${out}mergelist | awk -F "_" '{print $4}' | awk -F "/" '{print $NF}')

for value in $mlist
do
cat /workspace/hramzr/2_Phd_PROJECT/cnv_calling/80_snapper_samples_generations | egrep $value
done


In [ ]:
import pandas as pd
indvs = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs", "r")
dict_vars={}
chroms=[]
for i in range(25):
    chroms.append("LG"+str(i+1))

for indv in indvs:
    vcf = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/"+ \
               str(indv.split()[0])+"/outputs/"+str(indv.split()[0])+"_q20.survivor_sorted.vcf", "r")
    for chrom in chroms:
        dict_vars[indv.strip()]={chrom:{'DEL':0, 'DUP':0, 'INS':0, 'INV':0, 'TRA':0, 'BND':0}}
    print(dict_vars)

    for line in vcf:
        if "#" not in line:
            chrom=line.split(';')[0].split("\t")[0]
            var=line.split(';')[3][7:10]
            print(di)
            dict_vars[indv.strip()][chrom][var]+=1            
            
            
print(dict_vars['261931510'])

In [ ]:
import pandas as pd
females = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/females.lst", "r")
males = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/males.lst", "r")

df = pd.DataFrame([["chrom", ['bp_len']]])
for indv in females:
    try:
        vcf = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/"+ \
             str(indv.split()[0])+"/outputs/"+str(indv.split()[0])+"_q20.survivor_sorted.vcf", "r")
        print (indv)
        for line in vcf:
            if "#" not in line:
                 chrom=line.split(';')[0].split("\t")[0]
                 pos=line.split(';')[0].split("\t")[1]
                 var=line.split(';')[0].split("\t")[4]
                 length=line.split(';')[2].split("=")[1]
#                  print(chrom,pos,var)  
                 
#                  row_df = pd.DataFrame([chrom,length])
#                  df = pd.concat([row_df, df], ignore_index=True)
#                  print (chrom,pos,var,length)
                
    except:
        pass
    

In [ ]:
echo "----male-----"
cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/*/outputs/*_q20.survivor_sorted.vcf | egrep -v "#" | \
tr ";" " " | awk '{print substr($3,1,3)}' | sort | uniq

# echo "----male2-----"
# cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/275378212/outputs/275378212_q20.survivor_sorted.vcf | egrep -v "#" | \
# tr ";" " " | awk '{print $1,$2,substr($3,1,3), substr($10,8)}' | awk '{print $3}' | sort | uniq -c

# echo "----male3-----"
# cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/734562795/outputs/734562795_q20.survivor_sorted.vcf | egrep -v "#" | \
# tr ";" " " | awk '{print $1,$2,substr($3,1,3), substr($10,8)}' | awk '{print $3}' | sort | uniq -c


# echo "----male4-----"
# cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/261931510/outputs/261931510_q20.survivor_sorted.vcf | egrep -v "#" | \
# tr ";" " " | awk '{print $1,$2,substr($3,1,3), substr($10,8)}' | awk '{print $3}' | sort | uniq -c

# echo "----female-----"

# cat /workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/299826357/outputs/299826357_q20.survivor_sorted.vcf | egrep -v "#" | \
# tr ";" " " | awk '{print $1,$2,substr($3,1,3), substr($10,8)}' | awk '{print $3}' | sort | uniq -c